ResNet50 model 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm as notebook_tqdm
from datetime import datetime
import numpy as np 
import pandas as pd 

from PIL import Image
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from tqdm import tqdm
from PIL import Image
import torch
import torchvision
from torchvision.models.resnet import resnet50


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    torch.cuda.empty_cache()


/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd 
import numpy as np 



import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
import torchvision 
import torchvision.transforms as transforms 
from torchvision.transforms import ToTensor
from torchvision import datasets 
from datetime import datetime

In [3]:
transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [4]:
def get_train_valid_loader(
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           num_workers=2):

    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg


    # load the dataset

    base_dataset = datasets.ImageFolder(
        root='/volumes1/thesis/notebooks/data/gtsrb/GTSRB/Training', transform=transforms,
    )

    # TODO
    split_datasets = torch.utils.data.random_split(base_dataset, [0.20,0.8])
    global val_dataset 
    val_dataset = split_datasets[0]
    global train_dataset
    train_dataset = split_datasets[1]
    

    global num_train 
    num_train= len(train_dataset)
    indices = list(range(num_train))
    global split 
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)


    #train_idx, valid_idx = indices[split:], indices[:split]
    #train_sampler = SubsetRandomSampler(train_idx)
    #valid_sampler = SubsetRandomSampler(valid_idx)


    global train_loader 
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size,
        num_workers=num_workers, 
        #sampler = train_sampler
    )
    global valid_loader 
    valid_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size,
        num_workers=num_workers, 
        #sampler = valid_sampler
    )

    return train_loader, valid_loader

print(get_train_valid_loader(batch_size = 64, augment = True, random_seed = 1))


(<torch.utils.data.dataloader.DataLoader object at 0x7f72e04a1340>, <torch.utils.data.dataloader.DataLoader object at 0x7f7230c2ed00>)


In [5]:
# defining hyperparameters 
epochs = 10 
#epochs = 31   #the nn will train 31 times over all 50.000 images and validate itself with 12.000 images x 31  times) 
learning_rate = 0.0001 #how much the weight will be updated each time 
batch_size = 64 
classes = 43 
img_size = 32
random_seed = 42

In [6]:
classes = [ '0:Speed limit (20km/h)',
            '1:Speed limit (30km/h)', 
            '2:Speed limit (50km/h)', 
            '3:Speed limit (60km/h)', 
            '4:Speed limit (70km/h)', 
            '5:Speed limit (80km/h)', 
            '6:End of speed limit (80km/h)', 
            '7:Speed limit (100km/h)', 
            '8:Speed limit (120km/h)', 
            '9:No passing', 
            '10:No passing veh over 3.5 tons', 
            '11:Right-of-way at intersection', 
            '12:Priority road', 
            '13:Yield', 
            '14:Stop', 
            '15:No vehicles', 
            '16:Veh > 3.5 tons prohibited', 
            '17:No entry', 
            '18:General caution', 
            '19:Dangerous curve left', 
            '20:Dangerous curve right', 
            '21:Double curve', 
            '22:Bumpy road', 
            '23:Slippery road', 
            '24:Road narrows on the right', 
            '25:Road work', 
            '26:Traffic signals', 
            '27:Pedestrians', 
            '28:Children crossing', 
            '29:Bicycles crossing', 
            '30:Beware of ice/snow',
            '31:Wild animals crossing', 
            '32:End speed + passing limits', 
            '33:Turn right ahead', 
            '34:Turn left ahead', 
            '35:Ahead only', 
            '36:Go straight or right', 
            '37:Go straight or left', 
            '38:Keep right', 
            '39:Keep left', 
            '40:Roundabout mandatory', 
            '41:End of no passing', 
            '42:End no passing veh > 3.5 tons' ]

In [7]:
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

In [8]:
def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    #plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')

In [9]:
# train function

def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    model.train()
    running_loss = 0
    
    for X, y_true in train_loader:
        benign_x = X.cpu().numpy()
        optimizer.zero_grad()
        
        y_true = y_true.to(device)

        y_hat = torch.argmax(model(torch.Tensor(x_adversarial))) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

In [10]:
# validation function, without a learning step (backward pass)

def validate(val_dataset, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in val_dataset:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat= model(X)    # predicted
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
        
    return model, epoch_loss

In [11]:
# training function
def training_loop(model, criterion, optimizer, train_dataset, val_dataset, epochs, device, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_dataset, model, criterion, optimizer, device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(val_dataset, model, criterion, device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):
            
            train_acc = get_accuracy(model, train_dataset, device=device)
            valid_acc = get_accuracy(model, val_dataset, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')

    plot_losses(train_losses, valid_losses)
    
    return model, optimizer, (train_losses, valid_losses)

In [12]:
model = resnet50(pretrained=False)
model.fc = torch.nn.Linear(2048,43)
model.conv1 = torch.nn.Conv2d(3,64,kernel_size=5,stride=1)

#model = model.to('cpu')
model = model.to(device)


In [13]:

torch.manual_seed(random_seed)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [16]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_dataset, val_dataset, epochs, device)

NameError: name 'FastGradientMethod' is not defined

# evaluation 

In [148]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

In [59]:
print("Model's state dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
print("")

Model's state dict:
conv1.weight 	 torch.Size([64, 3, 5, 5])
conv1.bias 	 torch.Size([64])
bn1.weight 	 torch.Size([64])
bn1.bias 	 torch.Size([64])
bn1.running_mean 	 torch.Size([64])
bn1.running_var 	 torch.Size([64])
bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv1.weight 	 torch.Size([64, 64, 1, 1])
layer1.0.bn1.weight 	 torch.Size([64])
layer1.0.bn1.bias 	 torch.Size([64])
layer1.0.bn1.running_mean 	 torch.Size([64])
layer1.0.bn1.running_var 	 torch.Size([64])
layer1.0.bn1.num_batches_tracked 	 torch.Size([])
layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight 	 torch.Size([64])
layer1.0.bn2.bias 	 torch.Size([64])
layer1.0.bn2.running_mean 	 torch.Size([64])
layer1.0.bn2.running_var 	 torch.Size([64])
layer1.0.bn2.num_batches_tracked 	 torch.Size([])
layer1.0.conv3.weight 	 torch.Size([256, 64, 1, 1])
layer1.0.bn3.weight 	 torch.Size([256])
layer1.0.bn3.bias 	 torch.Size([256])
layer1.0.bn3.running_mean 	 torch.Size([256])
layer1.0.bn3.running_var 	 torc

In [62]:
MODEL_FOLDER = "../Model"
if not os.path.isdir(MODEL_FOLDER):
    os.mkdir(MODEL_FOLDER)
    
PATH_TO_MODEL = MODEL_FOLDER + "/pytorch_classification_resnet50.pth"
if os.path.exists(PATH_TO_MODEL):
    os.remove(PATH_TO_MODEL)
torch.save(model.state_dict(), PATH_TO_MODEL)

print("Model saved at %s" %(PATH_TO_MODEL))

Model saved at ../Model/pytorch_classification_resnet50.pth


In [60]:
from torchsummary import summary
print(summary(model, (3, 32, 32))) 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]           4,864
       BatchNorm2d-2           [-1, 64, 28, 28]             128
              ReLU-3           [-1, 64, 28, 28]               0
         MaxPool2d-4           [-1, 64, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]           4,096
       BatchNorm2d-6           [-1, 64, 14, 14]             128
              ReLU-7           [-1, 64, 14, 14]               0
            Conv2d-8           [-1, 64, 14, 14]          36,864
       BatchNorm2d-9           [-1, 64, 14, 14]             128
             ReLU-10           [-1, 64, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]          16,384
      BatchNorm2d-12          [-1, 256, 14, 14]             512
           Conv2d-13          [-1, 256, 14, 14]          16,384
      BatchNorm2d-14          [-1, 256,

In [17]:
classes = 43
num = range(classes)
labels = []
for i in num:
    labels.append(str(i))
labels = sorted(labels)
for i in num:
    labels[i] = int(labels[i])
print("List of labels : ")
print("Actual labels \t--> Class in PyTorch")
for i in num:
    print("\t%d \t--> \t%d" % (labels[i], i))

List of labels : 
Actual labels 	--> Class in PyTorch
	0 	--> 	0
	1 	--> 	1
	10 	--> 	2
	11 	--> 	3
	12 	--> 	4
	13 	--> 	5
	14 	--> 	6
	15 	--> 	7
	16 	--> 	8
	17 	--> 	9
	18 	--> 	10
	19 	--> 	11
	2 	--> 	12
	20 	--> 	13
	21 	--> 	14
	22 	--> 	15
	23 	--> 	16
	24 	--> 	17
	25 	--> 	18
	26 	--> 	19
	27 	--> 	20
	28 	--> 	21
	29 	--> 	22
	3 	--> 	23
	30 	--> 	24
	31 	--> 	25
	32 	--> 	26
	33 	--> 	27
	34 	--> 	28
	35 	--> 	29
	36 	--> 	30
	37 	--> 	31
	38 	--> 	32
	39 	--> 	33
	4 	--> 	34
	40 	--> 	35
	41 	--> 	36
	42 	--> 	37
	5 	--> 	38
	6 	--> 	39
	7 	--> 	40
	8 	--> 	41
	9 	--> 	42


In [61]:
df = pd.read_csv("/volumes1/thesis/notebooks/data/Test.csv")
numExamples = len(df)
labels_list = list(df.ClassId)

In [79]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1):

In [20]:
# loading dataset 
training_dataset = torchvision.datasets.GTSRB(
    root='./data', split = 'train', transform=transforms, download=True)


transforms = transforms.Compose([transforms.Resize((32, 32)),
    transforms.ToTensor()])
    
test_dataset = torchvision.datasets.GTSRB(
    root='./data', split = 'test', transform=transforms, download=True)

# train loader 
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size
, shuffle=False, num_workers=2)

In [17]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier

# FGSM Attack 

In [22]:
X, y = test_dataset[0]
X.shape, X.min(), X.max()

# class NormalizeInverse(torchvision.transforms.Normalize):
#     """
#     Undoes the normalization and returns the reconstructed images in the input domain.
#     """

#     def __init__(self, mean, std):
#         mean = torch.as_tensor(mean)
#         std = torch.as_tensor(std)
#         std_inv = 1 / (std + 1e-7)
#         mean_inv = -mean * std_inv
#         super().__init__(mean=mean_inv, std=std_inv)

#     def __call__(self, tensor):
#         return super().__call__(tensor.clone())


# inverse_normalize = NormalizeInverse(mean=(0.3403, 0.3121, 0.3214), std=(0.2724, 0.2608, 0.2669))  # Same mean and std values as defined in the original transforms.
# #torchvision.transforms.ToPILImage()((X))

class WrappedModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.trained_model = model  # "model" is our trained model
    self.trained_model.to("cpu")
    self.normalize = torchvision.transforms.Normalize((0.3403, 0.3121, 0.3214), (0.2724, 0.2608, 0.2669))

  def forward(self, x: torch.Tensor):
    # x comes in as a Tensor of shape [3, 32, 32]
    # Manually add a batch dimension
    x = torch.unsqueeze(x, dim=0)
    # Shape of x is now [1, 3, 32, 32]
    # Normalize the input
    x_normalized = self.normalize(x)
    # Return only the logits
    logits = self.trained_model(x_normalized)
    return logits

# We again wrap our (already wrapped) model in a PyTorchClassifier, which ART knows how to use
classifier = PyTorchClassifier(
      model=WrappedModel(),
      clip_values=(0.0, 1.0),  # The adversarial sample tensor values will be within these values
      loss=criterion,  # defined above
      optimizer=optimizer,  # defined above
      input_shape=(3, 32, 32),
      nb_classes=43,
      device_type="cpu"
)

attack = FastGradientMethod(estimator=classifier, eps=0.1)
# benign_x = inverse_normalize(X)
benign_x = X.cpu().numpy()  # convert torch to numpy
type(benign_x), benign_x.min(), benign_x.max()
x_adversarial = attack.generate(benign_x)

In [ ]:
import os 
#path = "/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm"
#os.mkdir(path)
test_model = WrappedModel()
attack = FastGradientMethod(estimator=classifier, eps=0.1)
for i, (x, y_true) in enumerate(test_dataset):
    benign_x = x.cpu().numpy()
    y_predicted = torch.argmax(test_model(torch.Tensor(benign_x)))
    if y_predicted == y_true:
        x_adversarial = attack.generate(benign_x)
        y_adversarial = torch.argmax(test_model(torch.Tensor(x_adversarial)))
        if y_adversarial != y_predicted:
            np.save(f"{path}/adv_samples_resnet50_fgsm_sample_{i}", x_adversarial)
    else:
        print('failed')

In [ ]:
import os 
#path = "/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm"
#os.mkdir(path)
test_model = WrappedModel()
attack = FastGradientMethod(estimator=classifier, eps=0.1)
pred_corr = []
pred_false = []
for i, (x, y_true) in enumerate(test_dataset):
    benign_x = x.cpu().numpy()  # sample
    y_predicted = torch.argmax(test_model(torch.Tensor(benign_x)))  #simple prediction
    if y_predicted == y_true:
        x_adversarial = attack.generate(benign_x)
        y_adversarial = torch.argmax(test_model(torch.Tensor(x_adversarial)))   # prediction on adv. sample 
        if y_adversarial != y_predicted:
            np.save(f"{path}/adv_samples_resnet50_fgsm_sample_{i}", x_adversarial)
    else:
        print('failed')

# CW Attack

In [28]:
from art.attacks.evasion import CarliniL0Method, CarliniL2Method, CarliniLInfMethod
from torchvision.datasets import GTSRB
from art.estimators.classification import PyTorchClassifier

class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.trained_model = model  # "model" is our trained model
        self.trained_model.to("cpu")
        self.normalize = torchvision.transforms.Normalize((0.3403, 0.3121, 0.3214), (0.2724, 0.2608, 0.2669))
    def forward(self, x: torch.Tensor):
        # x comes in as a Tensor of shape [1, 3, 32, 32] (same as the input size passed to attack.generate())
        # Normalize the input
        x_normalized = self.normalize(x)
        # Return only the logits
        logits = self.trained_model(x_normalized)
        return logits


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
wrapped_model = WrappedModel(model)
normalize = torchvision.transforms.Normalize((0.3403, 0.3121, 0.3214), (0.2724, 0.2608, 0.2669))
# We again wrap our (already wrapped) model in a PyTorchClassifier, which ART knows how to use
classifier = PyTorchClassifier(
    model=wrapped_model,
    # model=model,
    clip_values=(0.0, 1.0),  # The adversarial sample tensor values will be within these values
    loss=criterion,  # defined above
    optimizer=optimizer,  # defined above
    input_shape=(3, 32, 32),
    nb_classes=43,
    device_type="cpu",
    channels_first=True,
)
transforms = torchvision.transforms.Compose(
    [torchvision.transforms.Resize((32, 32)), torchvision.transforms.ToTensor()]
)

test_dataset = GTSRB(root="data", split="test", transform=transforms, download=True)
x, y_true = test_dataset[0]
# cw0_attack = CarliniL0Method(classifier, learning_rate=0.0001)
# l2_attack = CarliniL2Method(classifier, learning_rate=0.0001)
# linf_attack = CarliniLInfMethod(classifier, learning_rate=0.0001)
# art_input = np.expand_dims(x.numpy(), axis=0)
# adv_samples = cw0_attack.generate(art_input)
# adv_samples = l2_attack.generate(art_input)
# adv_samples = linf_attack.generate(art_input)


In [29]:
tested_model = wrapped_model.to('cpu')
tested_model

WrappedModel(
  (trained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 2

In [123]:
import os 
path = "/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm"
os.mkdir(path)
for i, (x, y_true) in enumerate(test_dataset):
    attack = FastGradientMethod(estimator=classifier, eps=0.1)
    benign_x = x.cpu().numpy()
    test_model = WrappedModel()
    y_predicted = torch.argmax(test_model(torch.Tensor(benign_x)))
    #type(benign_x), benign_x.min(), benign_x.max()
    if y_predicted == y_true:
        x_adversarial = attack.generate(benign_x)
        y_adversarial = torch.argmax(test_model(torch.Tensor(x_adversarial)))
        if y_adversarial != y_predicted:
            np.save(f"{path}/adv_samples_resnet50_fgsm_sample_{i}", x_adversarial)
    else:
        print('failed')

failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed

In [163]:
#path = "/volumes1/thesis/data/gtsrb/adv_samples_resnet50_l2"
#os.mkdir(path)
l2_attack = CarliniL2Method(classifier, learning_rate=0.0001)
for i, (x, y_true) in enumerate(test_dataset):
    art_input = np.expand_dims(x.numpy(), axis=0)
    y_predicted = torch.argmax(tested_model(torch.Tensor(art_input)))
    if y_predicted == y_true:
        adv_samples = l2_attack.generate(art_input) 
        y_adversarial = torch.argmax(tested_model(torch.Tensor(adv_samples)))
        if y_adversarial != y_predicted:
            np.save(f"{path}/adv_samples_resnet50_l2_sample_{i}", adv_samples)
        else: 
            print('failed')

C&W L_2: 100%|██████████| 1/1 [00:15<00:00, 15.70s/it]


failed


C&W L_2: 100%|██████████| 1/1 [00:15<00:00, 15.84s/it]


failed


C&W L_2: 100%|██████████| 1/1 [00:15<00:00, 15.48s/it]


...

failed


C&W L_2: 100%|██████████| 1/1 [00:16<00:00, 16.84s/it]


failed


C&W L_2:   0%|          | 0/1 [00:15<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
path = "/volumes1/thesis/data/gtsrb/adv_samples_resnet50_linf"
os.mkdir(path)
linf_attack = CarliniLInfMethod(classifier, learning_rate=0.0001)
for i, (x, y_true) in enumerate(test_dataset):
    art_input = np.expand_dims(x.numpy(), axis=0)
    y_predicted = torch.argmax(tested_model(torch.Tensor(art_input)))
    if y_predicted == y_true:
        adv_samples = linf_attack.generate(art_input) 
        y_adversarial = torch.argmax(tested_model(torch.Tensor(adv_samples)))
        if y_adversarial != y_predicted:
            np.save(f"{path}/adv_samples_resnet50_linf_sample_{i}", adv_samples)
        else: 
            print('failed')

# Training 


In [1]:
import numpy as np 
from PIL import Image   

In [ ]:
for i, (x, y) in enumerate(/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm):
    img_array = np.load("/adv_samples_resnet50_fgsm_sample_{i}")
    img_array = np.moveaxis(img_array, 0, 2)
    img_array = (img_array * 255).astype(np.uint8)
    img = Image.fromarray(img_array, "RGB")


In [102]:
from pathlib import Path 

In [106]:
Path('Adv_samples_resnet50_fgsm').absolute()

PosixPath('/volumes1/thesis/Adv_samples_resnet50_fgsm')

In [91]:
def npy_loader(path):
    return torch.from_numpy(np.load(path))

In [125]:

def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    img_array = np.moveaxis(sample, 0, 2)
    img_array = (img_array * 255).astype(np.uint8)
    img = Image.fromarray(img_array, "RGB")
    return img
    
dataset = datasets.DatasetFolder(
    root='/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm',
    loader=npy_loader,
    extensions=['.npy']
)

FileNotFoundError: Couldn't find any class folder in /volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm.

In [133]:
from torchvision.datasets import DatasetFolder
from torchvision import find_classes

ImportError: cannot import name 'find_classes' from 'torchvision' (/volumes1/thesis/notebooks/notebook_env/lib/python3.8/site-packages/torchvision/__init__.py)

In [130]:
find_classes('/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm')

NameError: name 'find_classes' is not defined

In [127]:
dataset = datasets.ImageFolder(
    root='/volumes1/thesis/data/gtsrb/Adv_samples_resnet50_fgsm',
    loader=npy_loader,
    extensions=['.npy']
)

TypeError: __init__() got an unexpected keyword argument 'extensions'

In [34]:
import os

import torch
import torch.optim as optim
import numpy as np

from torch import nn
from sklearn.utils import shuffle

from art.estimators.certification import deep_z
from art.utils import to_categorical

In [29]:
classifier = PyTorchClassifier(
    model=wrapped_model,
    # model=model,
    clip_values=(0.0, 1.0),  # The adversarial sample tensor values will be within these values
    loss=criterion,  # defined above
    optimizer=optimizer,  # defined above
    input_shape=(3, 32, 32),
    nb_classes=43,
    device_type="cpu",
    channels_first=True,
)

In [40]:
model.to('cpu')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1):

In [35]:
print(optimizer, criterion) 

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
) CrossEntropyLoss()
